# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#%matplotlib widget
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cleaned_weather_df=pd.read_csv('../output_data/cities.csv')
cleaned_weather_df.drop(axis=1, labels='Unnamed: 0', inplace=True)
cleaned_weather_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,saint-philippe,RE,-21.36,55.77,68.00,56,75,24.16,1595523865
1,provideniya,RU,64.38,-173.30,49.95,66,44,7.47,1595524018
2,khatanga,RU,71.97,102.50,42.40,85,100,17.92,1595524018
3,itoman,JP,26.12,127.67,82.40,88,40,10.29,1595523911
4,ushuaia,AR,-54.80,-68.30,39.20,60,90,5.82,1595523859
5,puerto ayora,EC,-0.74,-90.35,75.20,73,20,25.28,1595524019
6,listvyagi,RU,53.68,86.95,58.44,83,75,1.88,1595524019
7,college,US,64.86,-147.80,53.60,81,75,2.55,1595523897
8,hilo,US,19.73,-155.09,73.40,78,75,5.82,1595524019
9,nanortalik,GL,60.14,-45.24,47.44,91,100,5.48,1595523886


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Create df for lat long locations
locations = cleaned_weather_df[["Latitude", "Longitude"]].astype(float)

#Create series for humidity
humidity=cleaned_weather_df['Humidity'].astype(float)

# Create and add humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)

#Display heatmap
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
ideal_weather_df=cleaned_weather_df

#A max temperature lower than 80 degrees but higher than 70.
#ideal_weather_df=ideal_weather_df[ideal_weather_df['Max Temp']>70]
#ideal_weather_df=ideal_weather_df[ideal_weather_df['Max Temp']<80]

#Wind speed less than 10 mph.
#ideal_weather_df=ideal_weather_df[ideal_weather_df['Wind Speed']<10]

#Cloudiness less than 5.
#ideal_weather_df=ideal_weather_df[ideal_weather_df['Cloudiness']<5]

#Display ideal weather df
#ideal_weather_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=ideal_weather_df
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,saint-philippe,RE,-21.36,55.77,68.00,56,75,24.16,1595523865,
1,provideniya,RU,64.38,-173.30,49.95,66,44,7.47,1595524018,
2,khatanga,RU,71.97,102.50,42.40,85,100,17.92,1595524018,
3,itoman,JP,26.12,127.67,82.40,88,40,10.29,1595523911,
4,ushuaia,AR,-54.80,-68.30,39.20,60,90,5.82,1595523859,
5,puerto ayora,EC,-0.74,-90.35,75.20,73,20,25.28,1595524019,
6,listvyagi,RU,53.68,86.95,58.44,83,75,1.88,1595524019,
7,college,US,64.86,-147.80,53.60,81,75,2.55,1595523897,
8,hilo,US,19.73,-155.09,73.40,78,75,5.82,1595524019,
9,nanortalik,GL,60.14,-45.24,47.44,91,100,5.48,1595523886,


In [6]:
url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
params={
    'radius':5000,
    'type': 'lodging',
    'key': g_key
}

In [7]:
for index, row in hotel_df.iterrows():
    try:
        response=requests.get(url+str(hotel_df['Latitude'].iloc[index])+','+str(hotel_df['Longitude'].iloc[index]),params=params).json()
        hotel_df['Hotel Name'].iloc[index]=response['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].iloc[index]=None
        

C:\Users\dbayn\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
hotel_df.dropna(inplace=True)

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
\n
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Create df for lat long locations
locations = hotel_df[["Latitude", "Longitude"]].astype(float)

#Create series for humidity
humidity=hotel_df['Humidity'].astype(float)

# Create and add humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)


# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)
#markers = gmaps.marker_layer(locations=locations, info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_df['Hotel Name']])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))